In [1]:
import pandas as pd
import deepmatcher as dm
import os

In [2]:
hybrid_model = dm.MatchingModel(attr_summarizer='hybrid')
hybrid_model.load_state('../../models/hybrid1.pth')

In [5]:
#k is the number of neighbours to consider
def generateExplanations(nearest_neighbours,k,dataset,attribute,model,testset_path,true_label,temp_path='temp'):
    nearest_neighbours_values = pd.merge(nearest_neighbours,dataset,left_on=attribute,right_on='id')
    selected_cols = nearest_neighbours_values[['ltable_'+attribute,'rtable_'+attribute]]
    most_frequents_values = selected_cols.groupby(['ltable_'+attribute,'rtable_'+attribute]).size().reset_index(name='Time').sort_values(by=['Time'],ascending=False).reset_index()
    if (k>= most_frequents_values.shape[0]):
        k = most_frequents_values.shape[0]
    testset = dm.data.process_unlabeled(testset_path,model,ignore_columns=['id','label'])
    standard_preds = model.run_prediction(testset)
    if (true_label == 1):
        true_positives = standard_preds[standard_preds['match_score']>0.5]
    else:
        true_positives = standard_preds[standard_preds['match_score']<=0.5]
    print("The standard true positives are {}".format(str(true_positives.shape[0])))
    for i in range(k):
        testset = pd.read_csv(testset_path)
        testset['ltable_'+attribute] = most_frequents_values.loc[i]['ltable_'+attribute]
        testset['rtable_'+attribute] = most_frequents_values.loc[i]['rtable_'+attribute]
        altered_testName = 'altered_test'+str(i)+'.csv'
        testset.to_csv(os.path.join(temp_path,altered_testName),index=False)
        altered_test = dm.data.process_unlabeled(os.path.join(temp_path,altered_testName),model,ignore_columns = ['id','label'])
        altered_pred = model.run_prediction(altered_test)
        if (true_label == 1):
            true_positives_foraltered = altered_pred[altered_pred['match_score'] >0.5]
        else:
            true_positives_foraltered = altered_pred[altered_pred['match_score']<=0.5]
        print(true_positives_foraltered.shape[0])
    return most_frequents_values

In [ ]:
nn = pd.read_csv('experiments-results/exp3/testpos_nn.csv')
dataset = pd.read_csv('../../Structured/itunes-amazon/negatives.csv')
df = generateExplanations(nn,5,dataset,'Album_Name',hybrid_model,"../../Structured/itunes-amazon/positives.csv",1)

===>  PREDICT Epoch 9


In [3]:
altered_test = dm.data.process_unlabeled("temp/altered_test3.csv",hybrid_model,ignore_columns = ['id','label'])
altered_pred = hybrid_model.run_prediction(altered_test)
true_label=1
if (true_label == 1):
    true_positives_foraltered = altered_pred[altered_pred['match_score'] >0.5]
else:
    true_positives_foraltered = altered_pred[altered_pred['match_score']<=0.5]
print(true_positives_foraltered.shape[0])

===>  PREDICT Epoch 9
Finished Epoch 9 || Run Time:    0.8 | Load Time:    0.4 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00

118


In [12]:
df

,index,ltable_Price,rtable_Price,Time
0,4,Album Only,$ 1.29,14
1,0,$ 1.29,$ 0.89,9
2,3,$ 1.99,$ 1.29,2
3,1,$ 1.29,$ 0.99,1
4,2,$ 1.29,$ 1.29,1


In [10]:
alt = pd.read_csv('temp/altered_test0.csv')
alt.head()

,label,ltable_Song_Name,ltable_Artist_Name,ltable_Album_Name,ltable_Genre,ltable_Price,ltable_CopyRight,ltable_Time,ltable_Released,rtable_Song_Name,rtable_Artist_Name,rtable_Album_Name,rtable_Genre,rtable_Price,rtable_CopyRight,rtable_Time,rtable_Released,id
0,1,Silver and Gold,Little Big Town,Pain Killer,"Country , Music , Honky Tonk , Contemporary Co...",Album Only,"2014 Little Big Town , LLC Under exclusive lic...",3:33,21-Oct-14,Silver And Gold,Little Big Town,Pain Killer,Country,$ 1.29,( C ) 2014 Little Big Town LLC Under exclusive...,3:31,"October 21 , 2014",102
1,1,Halcyon,Ellie Goulding,Halcyon Days ( Deluxe Edition ),"Pop , Music , Dance , Rock",Album Only,‰ ãÑ 2013 Polydor Ltd. ( UK ),3:24,27-Aug-13,Halcyon,Ellie Goulding,Halcyon,Pop,$ 1.29,( C ) 2013 Polydor Ltd. ( UK ),3:24,"August 27 , 2013",308
2,1,True Colors ( feat . Nicki Minaj ),Wiz Khalifa,Blacc Hollywood,"Hip-Hop/Rap , Music , Rap , East Coast Rap , H...",Album Only,2014 Atlantic Recording Corporation for the Un...,4:15,19-Aug-14,True Colors ( feat . Nicki Minaj ) [ Explicit ],Wiz Khalifa,Blacc Hollywood [ Explicit ],Rap & Hip-Hop,$ 1.29,2014 Atlantic Recording Corporation for the Un...,4:15,"August 19 , 2014",35
3,1,"Turn Around ( 5,4,3,2,1 ) [ DJ Bam Bam Radio R...",Flo Rida,"Turn Around ( 5,4,3,2,1 ) - Deluxe Single","Hip-Hop/Rap , Music , Electronic , Rap , Dirty...",Album Only,‰ ãÑ 2010 Atlantic Recording Corporation for t...,3:35,16-Nov-10,"Turn Around ( 5,4,3,2,1 )",Flo Rida,Only One Flo ( Part 1 ) [ Deluxe ] [ + Video ]...,Rap & Hip-Hop,$ 1.29,2010 Atlantic Recording Corporation for the Un...,3:20,"November 26 , 2010",9
4,1,Gone,The Weeknd,Trilogy,"R&B / Soul , Music , Rock , Contemporary R&B",Album Only,"‰ ãÑ 2012 XO & co. , Inc.",8:07,13-Nov-12,Gone ( Album Version ( Edited ) ) [ Clean ],The Weeknd,Trilogy ( Edited Version ) [ Clean ],R&B,$ 1.29,"( C ) 2013 XO & co. , Inc.",8:07,"September 18 , 2015",169


## Compare nearest neighbours with values distribution

In [1]:
## this function return occurrences in negatives and positives samples
def analyze_valueDistribution(dataset,value,column_provenance):
    dataset_pos = dataset[dataset['label']==1]
    dataset_neg = dataset[dataset['label']==0]
    posProvenanceDist = dict(dataset_pos[column_provenance].value_counts())
    negProvenanceDist = dict(dataset_neg[column_provenance].value_counts())
    if value in posProvenanceDist and value in negProvenanceDist:
        return negProvenanceDist[value],posProvenanceDist[value]
    elif value in negProvenanceDist:
        return negProvenanceDist[value],0
    elif value in posProvenanceDist:
        return 0,posProvenanceDist[value]
    else:
        return 0,0

In [9]:
train = pd.read_csv('../../Structured/itunes-amazon/merged_train.csv')
analyze_valueDistribution(train,"Album Only","ltable_Price")

(20, 2)

In [10]:
testpos_nn = pd.read_csv("experiments-results/exp3/testpos_nn.csv")
allNeg = pd.read_csv("../../Structured/itunes-amazon/negatives.csv")
nnValues = pd.merge(testpos_nn,allNeg,left_on="Album_Name",right_on="id")

In [20]:
def analyzeNearestNeighbours(training_set,nn_df,attribute,left_prefix='ltable_',right_prefix='rtable_'):
    nn_df["attribute"] = nn_df[left_prefix+attribute] + "|" +nn_df[right_prefix+attribute]
    attributeValues = list(nn_df["attribute"].value_counts().keys())
    for value in attributeValues:
        l_r_val = value.split("|")
        print (l_r_val[0])
        print (l_r_val[1])
        print (analyze_valueDistribution(training_set,l_r_val[0],left_prefix+attribute))
        print(analyze_valueDistribution(training_set,l_r_val[1],right_prefix+attribute))

In [23]:
analyzeNearestNeighbours(train,nnValues,"Price")

$ 1.29
$ 1.29
(177, 60)
(198, 63)
$ 0.99
$ 1.29
(20, 13)
(198, 63)
$ 1.29
$ 0.99
(177, 60)
(38, 14)
